<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/RobustMVP_Closed_Form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sympy.matrices.common import NonInvertibleMatrixError

In [34]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Exact robust mean–variance solver (model-specific Cᵐ)
— Matrix-int 加算バグ修正・substitution 完全化版
"""

from __future__ import annotations
import itertools, random
from fractions import Fraction
from typing import List, Dict, Optional
import sympy as sp
from sympy.matrices.common import NonInvertibleMatrixError

# ---------------- helper -------------------------------------------
def R(x):
    if isinstance(x, sp.Rational):        return x
    if isinstance(x, (int, sp.Integer)):  return sp.Rational(x)
    if isinstance(x, Fraction):           return sp.Rational(x.numerator, x.denominator)
    return sp.Rational(Fraction(str(x)).limit_denominator())

def fmt15(e: sp.Expr) -> str:
    return format(float(sp.N(e, 17)), ".15f")

# ---------------- 前提チェック -------------------------------------
def check_inputs(r, sigma, C, K, M):
    assert 2 <= M <= 4 and 2 <= K <= 5
    assert len(r) == len(sigma) == M
    assert all(len(v) == K for v in r)
    assert all(len(v) == K for v in sigma)
    if sp.Matrix(r).rank() < M:
        raise ValueError("r^m が線形独立でない")
    if any(any(x <= 0 for x in row) for row in sigma):
        raise ValueError("σ に非正が混入")
    if C:
        for Ci in C:
            Cmat = sp.Matrix([[R(x) for x in row] for row in Ci])
            if min(float(ev) for ev in Cmat.eigenvals()) <= 0:
                raise ValueError("C^m 非正定値")

# ---------------- Problem container --------------------------------
class ProblemData:
    def __init__(self, r, sigma, mu, C=None):
        self.M, self.K = len(r), len(r[0])
        check_inputs(r, sigma, C, self.K, self.M)
        self.mu = R(mu)
        self.r  = [sp.Matrix([R(x) for x in row]) for row in r]
        self.Vm = []
        for m in range(self.M):
            σ = sp.diag(*[R(x) for x in sigma[m]])
            Cmat = sp.Matrix([[R(x) for x in row] for row in C[m]]) if C else sp.eye(self.K)
            self.Vm.append(σ * Cmat * σ)

    def Vw(self, w):
        # ← sum() による Matrix+int エラーを回避するため明示ループ
        rbar = sp.zeros(self.K, 1)
        Vsum = sp.zeros(self.K, self.K)
        for m in range(self.M):
            rbar += w[m] * self.r[m]
            Vsum += w[m] * (self.Vm[m] + self.r[m] * self.r[m].T)
        return Vsum - rbar * rbar.T

    def pi_lambda_g(self, w_syms, S):
        V    = sp.simplify(self.Vw(w_syms))
        Vinv = sp.simplify(V.inv())
        R_S  = sp.Matrix.hstack(*[self.r[m] for m in S])
        one  = sp.Matrix([[1]]*len(S))
        M_inv= sp.simplify((R_S.T*Vinv*R_S).inv())
        lam  = sp.simplify(2*self.mu*M_inv*one)
        pi   = sp.simplify(0.5*Vinv*R_S*lam)
        g    = sp.simplify((pi.T*V*pi)[0])
        return pi, lam, g

# ---------------- Outer solver -------------------------------------
class Solver:
    def __init__(self, data: ProblemData):
        self.D = data
        self.w  = sp.symbols(f"w1:{data.M}")
        self.wM = 1 - sum(self.w)
        self.W  = list(self.w) + [self.wM]
        self.boundary = self._make_boundary()

    def _make_boundary(self):
        pts=[]
        unit=[1]+[0]*(self.D.M-1)
        for p in itertools.permutations(unit): pts.append(list(map(sp.Integer,p)))
        for i,j in itertools.combinations(range(self.D.M),2):
            p=[0]*self.D.M; p[i]=p[j]=sp.Rational(1,2); pts.append(list(map(sp.Integer,p)))
        return pts

    @staticmethod
    def _feasible_numeric(w):
        fl=[float(sp.N(x)) for x in w]
        return all(v>=-1e-8 for v in fl) and abs(sum(fl)-1)<1e-8

    def _numeric_roots(self,eqs):
        roots=[]
        for _ in range(30):
            g0=[random.random() for _ in self.w]
            g0=[v/sum(g0)*0.9 for v in g0]
            try:
                sol=sp.nsolve(eqs,self.w,g0,tol=1e-28,maxsteps=100)
                sol=[sp.nsimplify(v,[sp.Rational(1,10000)]) for v in sol]
                if sol not in roots: roots.append(sol)
            except: pass
        return [{wi:sol[i] for i,wi in enumerate(self.w)} for sol in roots]

    # 共通可行性チェック
    def _candidate(self, w_vals, S, pi_sym, lam_sym, g_sym):
        if not self._feasible_numeric(w_vals): return None
        subs={wi:w_vals[i] for i,wi in enumerate(self.w)}
        subs[self.wM]=w_vals[-1]                       # ← wM も置換
        if min(float(sp.N(lam_sym[i,0].subs(subs))) for i in range(len(S)))<=1e-8: return None
        pi_num=[pi_sym[i].subs(subs) for i in range(self.D.K)]
        for m in range(self.D.M):
            if m not in S:
                lhs=float(sp.N(sum(pi_num[i]*self.D.r[m][i] for i in range(self.D.K))))
                if lhs < float(self.D.mu)-1e-12: return None
        g_val=g_sym.subs(subs)
        return {"g":sp.nsimplify(g_val),"w":w_vals,"S":S,"pi":pi_num}

    def solve(self)->Optional[Dict]:
        best=None
        # 内部点
        for k in range(1,self.D.K+1):
            for S in itertools.combinations(range(self.D.M),k):
                try: pi_s,lam_s,g_s=self.D.pi_lambda_g(self.W,S)
                except NonInvertibleMatrixError: continue
                grad=[sp.diff(g_s,wi).as_numer_denom()[0].cancel() for wi in self.w]
                try: sols=sp.solve(grad,list(self.w),dict=True,rational=True)
                except: sols=[]
                if not sols: sols=self._numeric_roots(grad)
                for sol in sols:
                    w_vals=[sp.nsimplify(sol.get(wi,0)) for wi in self.w]+[sp.nsimplify(self.wM.subs(sol))]
                    cand=self._candidate(w_vals,S,pi_s,lam_s,g_s)
                    if cand and (best is None or sp.N(cand["g"])>sp.N(best["g"])): best=cand
        # 境界点
        for p in self.boundary:
            w_vals=p
            subs_dict={wi:p[i] for i,wi in enumerate(self.w)}
            subs_dict[self.wM]=p[-1]
            for k in range(1,self.D.K+1):
                for S in itertools.combinations(range(self.D.M),k):
                    try: pi_s,lam_s,g_s=self.D.pi_lambda_g(self.W,S)
                    except NonInvertibleMatrixError: continue
                    cand=self._candidate(w_vals,S,pi_s,lam_s,g_s)
                    if cand and (best is None or sp.N(cand["g"])>sp.N(best["g"])): best=cand
        return best

# ------------------------------------------------------------------
# demo
# ------------------------------------------------------------------
if __name__=="__main__":
    r=[[0.015,0.03],[0.025,0.06]]
    sigma=[[0.175,0.35],[0.225,0.45]]
    C=[[[1,0.85],[0.85,1]],[[1,0.65],[0.65,1]]]
    mu=0.03

    res=Solver(ProblemData(r,sigma,mu,C)).solve()
    if res is None:
        print("★ 可行な活性集合が見つかりませんでした。μ を下げるかデータを見直してください。")
        exit()

    sp.init_printing()
    print("\n=== 厳密最適解 ===")
    print(f"活性集合 S : {res['S']}\n")

    print("最適 w:")
    for i,w_i in enumerate(res["w"],1):
        print(f"  w{i} = {w_i}  ≈ {fmt15(w_i)}")

    print("\nπ* :")
    for i,pi_i in enumerate(res["pi"],1):
        print(f"  π{i} = {pi_i}  ≈ {fmt15(pi_i)}")

    print(f"\n目的値 g(w*) = {res['g']}  ≈ {fmt15(res['g'])}")



=== 厳密最適解 ===
活性集合 S : (0,)

最適 w:
  w1 = 0  ≈ 0.000000000000000
  w2 = 1  ≈ 1.000000000000000

π* :
  π1 = 1.00000000000000  ≈ 1.000000000000000
  π2 = 0.500000000000000  ≈ 0.500000000000000

目的値 g(w*) = 2673/16000  ≈ 0.167062500000000


In [22]:
0.140795907942650*0.05+0.148010230143375*0.02

0.010000000000000002

In [23]:
0.140795907942650*0.02+0.148010230143375*0.05

0.01021642966602175